# Streaming Tool Calls with OpenAI (Python)

This notebook demonstrates **how a language model can decide to call external tools**
while **streaming its response token by token**.

We build the system step by step, starting from a single tool and ending with
a ChatGPT-style `ai(query)` function that:

- streams text output incrementally,
- detects tool calls issued by the model,
- executes Python functions **outside the model**,
- feeds the result back into the conversation,
- and continues streaming the final answer.

---

## Key ideas

- **Models do not execute code**  
  Tool calls are *requests*, not actions.

- **Tools are split in two layers**
  - a *schema* (`tools`) shown to the model
  - an *executor* (`TOOLS`) implemented in Python

- **Streaming and tool calls are independent**
  - streaming controls *how output arrives*
  - tool calls control *what the model asks for*

---

## What you will learn

1. How OpenAI represents tool calls internally
2. Why tool call IDs are mandatory
3. How to safely stream partial output
4. How to resume generation after tool execution
5. How to build a minimal agent loop without frameworks

This notebook is intentionally **low-level and explicit**
to make every step visible and debuggable.


In [1]:
"""
Environment setup:
- HTTP/HTTPS proxy (DWD)
- Load OpenAI API key
"""

import os
from dotenv import load_dotenv

# DWD proxy
os.environ["HTTP_PROXY"]  = "http://ofsquid.dwd.de:8080"
os.environ["HTTPS_PROXY"] = "http://ofsquid.dwd.de:8080"
os.environ["http_proxy"]  = os.environ["HTTP_PROXY"]
os.environ["https_proxy"] = os.environ["HTTPS_PROXY"]

# Load API key
load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "❌ OPENAI_API_KEY not found"

print("✅ Environment ready")


✅ Environment ready


In [2]:
"""
This function is pure Python.
The model never sees or executes this code.
"""

from datetime import datetime, timezone as dt_timezone
import pytz

def get_time(timezone: str) -> str:
    if timezone.upper() == "UTC":
        return datetime.now(dt_timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

    if timezone.upper() == "CET":
        return datetime.now(pytz.timezone("Europe/Berlin")).strftime(
            "%Y-%m-%d %H:%M:%S CET"
        )

    return f"Unsupported timezone: {timezone}"


In [3]:
# This JSON schema is the ONLY information about the tool
# that the model is allowed to see.

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Return the current time in a given timezone",
            "parameters": {
                "type": "object",
                "properties": {
                    "timezone": {
                        "type": "string",
                        "description": "Timezone name, e.g. UTC or CET"
                    }
                },
                "required": ["timezone"]
            }
        }
    }
]


In [4]:
"""
OpenAI decides whether to call a tool.
Tool calls are returned in a structured channel.
"""

from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You may call tools if helpful."},
        {"role": "user", "content": "What is the current time in CET?"}
    ],
    tools=tools,
    tool_choice="auto"
)

response

ChatCompletion(id='chatcmpl-CstZuKLANAhFE4ZoItllJQF6JnAjq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8mQalQlBn5oQCQ3axUPGtj91', function=Function(arguments='{"timezone":"CET"}', name='get_time'), type='function')]))], created=1767199390, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_29330a9688', usage=CompletionUsage(completion_tokens=15, prompt_tokens=73, total_tokens=88, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [5]:
from pprint import pprint

pprint(response.model_dump(), width=50)

{'choices': [{'finish_reason': 'tool_calls',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': [],
                          'audio': None,
                          'content': None,
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': [{'function': {'arguments': '{"timezone":"CET"}',
                                                       'name': 'get_time'},
                                          'id': 'call_8mQalQlBn5oQCQ3axUPGtj91',
                                          'type': 'function'}]}}],
 'created': 1767199390,
 'id': 'chatcmpl-CstZuKLANAhFE4ZoItllJQF6JnAjq',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_29330a9688',
 'usage': {'completion_tokens': 15,
           'completion_tokens_details': {'accepted_prediction_tokens': 0

In [6]:
"""
Execute the tool call and return the result to the model.
"""

import json
from IPython.display import Markdown, display

msg = response.choices[0].message

if msg.tool_calls:
    tool_call = msg.tool_calls[0]

    # 🔑 OpenAI returns arguments as a JSON string
    args = json.loads(tool_call.function.arguments)

    # Execute tool
    result = get_time(**args)

    display(
        Markdown(
            "<div style='color:gray; font-style:italic'>"
            f"Tool result: {result}"
            "</div>"
        )
    )

    # Feed result back to the model
    final = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "What is the current time in CET?"},
            msg,
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result
            }
        ]
    )

    display(Markdown(final.choices[0].message.content))
else:
    print("No tool call was made.")


<div style='color:gray; font-style:italic'>Tool result: 2025-12-31 17:43:11 CET</div>

The current time in Central European Time (CET) is 5:43 PM on December 31, 2025.

# Combine Streaming Response with Tool Call

In [7]:
def ai_followup(messages):
    buffer = ""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True,
    )

    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.get("content"):
            buffer += delta["content"]
            display(Markdown(buffer))

TOOLS = {
    "get_time": get_time
}

In [8]:
import json
from openai import OpenAI
from IPython.display import display, Markdown

client = OpenAI()

def ai(query: str):
    messages = [{"role": "user", "content": query}]
    _stream_chat(messages)

def _stream_chat(messages):
    buffer = ""
    display_handle = display(Markdown(buffer), display_id=True)

    # Tool-call state (must persist across streamed chunks)
    tool_call_id = None
    tool_name = None
    tool_args_json = ""

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        stream=True,
    )

    for chunk in resp:
        delta = chunk.choices[0].delta

        # --- 1) streamed text tokens ---
        if hasattr(delta, "content") and delta.content:
            buffer += delta.content
            display_handle.update(Markdown(buffer))

        # --- 2) streamed tool call ---
        if hasattr(delta, "tool_calls") and delta.tool_calls:
            tc = delta.tool_calls[0]   # assume single tool call

            # capture ID once
            if tool_call_id is None and tc.id:
                tool_call_id = tc.id

            # capture name
            if tc.function and tc.function.name:
                tool_name = tc.function.name

            # arguments may arrive incrementally
            if tc.function and tc.function.arguments:
                tool_args_json += tc.function.arguments

    # --- No tool requested → stop ---
    if tool_name is None:
        return

    # --- 3) Execute tool ---
    try:
        tool_args = json.loads(tool_args_json) if tool_args_json.strip() else {}
    except json.JSONDecodeError as e:
        raise ValueError(
            f"Tool arguments were not valid JSON:\n{tool_args_json}"
        ) from e

    if tool_name not in TOOLS:
        raise ValueError(f"Unknown tool requested by model: {tool_name}")

    tool_result = TOOLS[tool_name](**tool_args)

    # --- 4) Echo tool call back (WITH ID!) ---
    messages.append({
        "role": "assistant",
        "tool_calls": [{
            "id": tool_call_id,
            "type": "function",
            "function": {
                "name": tool_name,
                "arguments": tool_args_json
            }
        }]
    })

    # --- 5) Provide tool result ---
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call_id,
        "content": str(tool_result)
    })

    # --- 6) Continue conversation ---
    _stream_chat(messages)


In [9]:
ai("What is the date and time in Berlin?")

The date and time in Berlin is December 31, 2025, at 17:43:12 CET.

In [10]:
ai("Tell me about Goethe!")

Johann Wolfgang von Goethe (1749-1832) was a German writer, statesman, and scientist, widely regarded as one of the most significant figures in Western literature. His works span many genres, including poetry, drama, literature, and scientific writings. Here are key highlights about Goethe:

1. **Literary Contributions**:
   - **Faust**: Goethe's most famous work is "Faust," a tragic play that explores themes of ambition, desire, and the human condition. It follows the story of Faust, a scholar who makes a pact with the devil, trading his soul for knowledge and pleasure.
   - **The Sorrows of Young Werther**: This early novel, published in 1774, reflects the Sturm und Drang movement, emphasizing emotion over reason. It tells the story of Werther, a young artist who falls in love with a woman engaged to another man, leading to tragedy.

2. **Philosophy and Science**:
   - Goethe was not only a writer but also a thinker and a scientist. He contributed to various scientific fields, including color theory, where he challenged Newton's theories on light and color.
   - He also had an interest in botany, and his findings in morphology paved the way for later developments in biology.

3. **Influence**:
   - Goethe's work has had a profound influence on literature and philosophy, impacting writers and thinkers across Europe and beyond, including figures like Nietzsche, Freud, and Schiller.
   - He is considered a key figure in the Weimar Classicism movement, which emphasized the harmony of human and nature, blending classicist forms with romantic content.

4. **Legacy**:
   - Goethe's influence extends beyond literature; he shaped modern German language and culture. He is often celebrated in literary circles and his home city of Weimar features prominently in cultural tourism.
   - His ideas on aesthetics, nature, and human psychology continue to resonate, making him a lasting figure in intellectual history.

5. **Personal Life**:
   - Goethe was born in Frankfurt am Main and had a privileged upbringing. He studied law at university but soon pursued a career in writing and public service.
   - He had a complicated love life, with several important relationships that influenced his writings.

Goethe's life and work encapsulate a blend of art, science, and philosophy, making him a crucial figure in the cultural history of Europe.